# Create a restaurant recommendation system based on user preferences.

## 1. Data preprocessing

In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv(r"Dataset .csv")

In [3]:
df_temp = pd.read_csv(r"Dataset .csv")

In [4]:
df

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9546,5915730,Naml۱ Gurme,208,��stanbul,"Kemanke�� Karamustafa Pa��a Mahallesi, R۱ht۱m ...",Karak�_y,"Karak�_y, ��stanbul",28.977392,41.022793,Turkish,...,Turkish Lira(TL),No,No,No,No,3,4.1,Green,Very Good,788
9547,5908749,Ceviz A��ac۱,208,��stanbul,"Ko��uyolu Mahallesi, Muhittin ��st�_nda�� Cadd...",Ko��uyolu,"Ko��uyolu, ��stanbul",29.041297,41.009847,"World Cuisine, Patisserie, Cafe",...,Turkish Lira(TL),No,No,No,No,3,4.2,Green,Very Good,1034
9548,5915807,Huqqa,208,��stanbul,"Kuru�_e��me Mahallesi, Muallim Naci Caddesi, N...",Kuru�_e��me,"Kuru�_e��me, ��stanbul",29.034640,41.055817,"Italian, World Cuisine",...,Turkish Lira(TL),No,No,No,No,4,3.7,Yellow,Good,661
9549,5916112,A���k Kahve,208,��stanbul,"Kuru�_e��me Mahallesi, Muallim Naci Caddesi, N...",Kuru�_e��me,"Kuru�_e��me, ��stanbul",29.036019,41.057979,Restaurant Cafe,...,Turkish Lira(TL),No,No,No,No,4,4.0,Green,Very Good,901


In [5]:
def remove_special(text):
    if isinstance(text, str):
        return re.sub(r'[^\w\s]','', text)
    else:
        return text

In [6]:
df = df.applymap(remove_special)

C:\Users\mayank dandriyal\AppData\Local\Temp\ipykernel_3480\958035582.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(remove_special)


In [7]:
df

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,Third Floor Century City Mall Kalayaan Avenue ...,Century City Mall Poblacion Makati City,Century City Mall Poblacion Makati City Makati...,121.027535,14.565443,French Japanese Desserts,...,Botswana PulaP,Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,Little Tokyo 2277 Chino Roces Avenue Legaspi V...,Little Tokyo Legaspi Village Makati City,Little Tokyo Legaspi Village Makati City Makat...,121.014101,14.553708,Japanese,...,Botswana PulaP,Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat Edsa ShangriLa,162,Mandaluyong City,Edsa ShangriLa 1 Garden Way Ortigas Mandaluyon...,Edsa ShangriLa Ortigas Mandaluyong City,Edsa ShangriLa Ortigas Mandaluyong City Mandal...,121.056831,14.581404,Seafood Asian Filipino Indian,...,Botswana PulaP,Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,Third Floor Mega Fashion Hall SM Megamall Orti...,SM Megamall Ortigas Mandaluyong City,SM Megamall Ortigas Mandaluyong City Mandaluyo...,121.056475,14.585318,Japanese Sushi,...,Botswana PulaP,No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,Third Floor Mega Atrium SM Megamall Ortigas Ma...,SM Megamall Ortigas Mandaluyong City,SM Megamall Ortigas Mandaluyong City Mandaluyo...,121.057508,14.584450,Japanese Korean,...,Botswana PulaP,Yes,No,No,No,4,4.8,Dark Green,Excellent,229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9546,5915730,Naml۱ Gurme,208,stanbul,Kemanke Karamustafa Paa Mahallesi R۱ht۱m Cadde...,Karak_y,Karak_y stanbul,28.977392,41.022793,Turkish,...,Turkish LiraTL,No,No,No,No,3,4.1,Green,Very Good,788
9547,5908749,Ceviz Aac۱,208,stanbul,Kouyolu Mahallesi Muhittin st_nda Caddesi No 8...,Kouyolu,Kouyolu stanbul,29.041297,41.009847,World Cuisine Patisserie Cafe,...,Turkish LiraTL,No,No,No,No,3,4.2,Green,Very Good,1034
9548,5915807,Huqqa,208,stanbul,Kuru_eme Mahallesi Muallim Naci Caddesi No 56 ...,Kuru_eme,Kuru_eme stanbul,29.034640,41.055817,Italian World Cuisine,...,Turkish LiraTL,No,No,No,No,4,3.7,Yellow,Good,661
9549,5916112,Ak Kahve,208,stanbul,Kuru_eme Mahallesi Muallim Naci Caddesi No 64B...,Kuru_eme,Kuru_eme stanbul,29.036019,41.057979,Restaurant Cafe,...,Turkish LiraTL,No,No,No,No,4,4.0,Green,Very Good,901


In [8]:
df.isna().sum()

Restaurant ID           0
Restaurant Name         0
Country Code            0
City                    0
Address                 0
Locality                0
Locality Verbose        0
Longitude               0
Latitude                0
Cuisines                9
Average Cost for two    0
Currency                0
Has Table booking       0
Has Online delivery     0
Is delivering now       0
Switch to order menu    0
Price range             0
Aggregate rating        0
Rating color            0
Rating text             0
Votes                   0
dtype: int64

In [9]:
df.fillna("Unkown", inplace= True)

In [10]:
df

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,Third Floor Century City Mall Kalayaan Avenue ...,Century City Mall Poblacion Makati City,Century City Mall Poblacion Makati City Makati...,121.027535,14.565443,French Japanese Desserts,...,Botswana PulaP,Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,Little Tokyo 2277 Chino Roces Avenue Legaspi V...,Little Tokyo Legaspi Village Makati City,Little Tokyo Legaspi Village Makati City Makat...,121.014101,14.553708,Japanese,...,Botswana PulaP,Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat Edsa ShangriLa,162,Mandaluyong City,Edsa ShangriLa 1 Garden Way Ortigas Mandaluyon...,Edsa ShangriLa Ortigas Mandaluyong City,Edsa ShangriLa Ortigas Mandaluyong City Mandal...,121.056831,14.581404,Seafood Asian Filipino Indian,...,Botswana PulaP,Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,Third Floor Mega Fashion Hall SM Megamall Orti...,SM Megamall Ortigas Mandaluyong City,SM Megamall Ortigas Mandaluyong City Mandaluyo...,121.056475,14.585318,Japanese Sushi,...,Botswana PulaP,No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,Third Floor Mega Atrium SM Megamall Ortigas Ma...,SM Megamall Ortigas Mandaluyong City,SM Megamall Ortigas Mandaluyong City Mandaluyo...,121.057508,14.584450,Japanese Korean,...,Botswana PulaP,Yes,No,No,No,4,4.8,Dark Green,Excellent,229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9546,5915730,Naml۱ Gurme,208,stanbul,Kemanke Karamustafa Paa Mahallesi R۱ht۱m Cadde...,Karak_y,Karak_y stanbul,28.977392,41.022793,Turkish,...,Turkish LiraTL,No,No,No,No,3,4.1,Green,Very Good,788
9547,5908749,Ceviz Aac۱,208,stanbul,Kouyolu Mahallesi Muhittin st_nda Caddesi No 8...,Kouyolu,Kouyolu stanbul,29.041297,41.009847,World Cuisine Patisserie Cafe,...,Turkish LiraTL,No,No,No,No,3,4.2,Green,Very Good,1034
9548,5915807,Huqqa,208,stanbul,Kuru_eme Mahallesi Muallim Naci Caddesi No 56 ...,Kuru_eme,Kuru_eme stanbul,29.034640,41.055817,Italian World Cuisine,...,Turkish LiraTL,No,No,No,No,4,3.7,Yellow,Good,661
9549,5916112,Ak Kahve,208,stanbul,Kuru_eme Mahallesi Muallim Naci Caddesi No 64B...,Kuru_eme,Kuru_eme stanbul,29.036019,41.057979,Restaurant Cafe,...,Turkish LiraTL,No,No,No,No,4,4.0,Green,Very Good,901


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9551 entries, 0 to 9550
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant ID         9551 non-null   int64  
 1   Restaurant Name       9551 non-null   object 
 2   Country Code          9551 non-null   int64  
 3   City                  9551 non-null   object 
 4   Address               9551 non-null   object 
 5   Locality              9551 non-null   object 
 6   Locality Verbose      9551 non-null   object 
 7   Longitude             9551 non-null   float64
 8   Latitude              9551 non-null   float64
 9   Cuisines              9551 non-null   object 
 10  Average Cost for two  9551 non-null   int64  
 11  Currency              9551 non-null   object 
 12  Has Table booking     9551 non-null   object 
 13  Has Online delivery   9551 non-null   object 
 14  Is delivering now     9551 non-null   object 
 15  Switch to order menu 

In [12]:
df.isna().sum()

Restaurant ID           0
Restaurant Name         0
Country Code            0
City                    0
Address                 0
Locality                0
Locality Verbose        0
Longitude               0
Latitude                0
Cuisines                0
Average Cost for two    0
Currency                0
Has Table booking       0
Has Online delivery     0
Is delivering now       0
Switch to order menu    0
Price range             0
Aggregate rating        0
Rating color            0
Rating text             0
Votes                   0
dtype: int64

In [13]:
df.duplicated(subset= "Cuisines").sum()

7725

In [14]:
df = df.drop_duplicates(subset= "Cuisines")

In [15]:
df.duplicated(subset= "Restaurant Name").sum()

74

In [16]:
df

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,Third Floor Century City Mall Kalayaan Avenue ...,Century City Mall Poblacion Makati City,Century City Mall Poblacion Makati City Makati...,121.027535,14.565443,French Japanese Desserts,...,Botswana PulaP,Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,Little Tokyo 2277 Chino Roces Avenue Legaspi V...,Little Tokyo Legaspi Village Makati City,Little Tokyo Legaspi Village Makati City Makat...,121.014101,14.553708,Japanese,...,Botswana PulaP,Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat Edsa ShangriLa,162,Mandaluyong City,Edsa ShangriLa 1 Garden Way Ortigas Mandaluyon...,Edsa ShangriLa Ortigas Mandaluyong City,Edsa ShangriLa Ortigas Mandaluyong City Mandal...,121.056831,14.581404,Seafood Asian Filipino Indian,...,Botswana PulaP,Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,Third Floor Mega Fashion Hall SM Megamall Orti...,SM Megamall Ortigas Mandaluyong City,SM Megamall Ortigas Mandaluyong City Mandaluyo...,121.056475,14.585318,Japanese Sushi,...,Botswana PulaP,No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,Third Floor Mega Atrium SM Megamall Ortigas Ma...,SM Megamall Ortigas Mandaluyong City,SM Megamall Ortigas Mandaluyong City Mandaluyo...,121.057508,14.584450,Japanese Korean,...,Botswana PulaP,Yes,No,No,No,4,4.8,Dark Green,Excellent,229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9542,5926979,Leman K_lt_r,208,stanbul,Caferaa Mahallesi Neet _mer Sokak No 9A Kad۱k_...,Kad۱k_y Merkez,Kad۱k_y Merkez stanbul,29.022805,40.989705,Restaurant Cafe,...,Turkish LiraTL,No,No,No,No,3,3.7,Yellow,Good,506
9544,5915547,Karak_y G_ll_olu,208,stanbul,Kemanke Karamustafa Paa Mahallesi R۱ht۱m Cadde...,Karak_y,Karak_y stanbul,28.977636,41.022904,Desserts B_rek,...,Turkish LiraTL,No,No,No,No,2,4.7,Dark Green,Excellent,1305
9545,5915054,Baltazar,208,stanbul,Kemanke Karamustafa Paa Mahallesi K۱l۱_ Ali Pa...,Karak_y,Karak_y stanbul,28.981103,41.025785,Burger Izgara,...,Turkish LiraTL,No,No,No,No,3,4.3,Green,Very Good,870
9547,5908749,Ceviz Aac۱,208,stanbul,Kouyolu Mahallesi Muhittin st_nda Caddesi No 8...,Kouyolu,Kouyolu stanbul,29.041297,41.009847,World Cuisine Patisserie Cafe,...,Turkish LiraTL,No,No,No,No,3,4.2,Green,Very Good,1034


In [17]:
def clean_text(city):
    result = str(city).lower()
    return(result.replace(' ',''))

In [18]:
df["City"] = df["City"].apply(clean_text)

C:\Users\mayank dandriyal\AppData\Local\Temp\ipykernel_3480\1852667530.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["City"] = df["City"].apply(clean_text)


In [19]:
df["Restaurant Name"] = df["Restaurant Name"].apply(clean_text)

C:\Users\mayank dandriyal\AppData\Local\Temp\ipykernel_3480\1697931787.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Restaurant Name"] = df["Restaurant Name"].apply(clean_text)


In [20]:
df

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,lepetitsouffle,162,makaticity,Third Floor Century City Mall Kalayaan Avenue ...,Century City Mall Poblacion Makati City,Century City Mall Poblacion Makati City Makati...,121.027535,14.565443,French Japanese Desserts,...,Botswana PulaP,Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,izakayakikufuji,162,makaticity,Little Tokyo 2277 Chino Roces Avenue Legaspi V...,Little Tokyo Legaspi Village Makati City,Little Tokyo Legaspi Village Makati City Makat...,121.014101,14.553708,Japanese,...,Botswana PulaP,Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,heatedsashangrila,162,mandaluyongcity,Edsa ShangriLa 1 Garden Way Ortigas Mandaluyon...,Edsa ShangriLa Ortigas Mandaluyong City,Edsa ShangriLa Ortigas Mandaluyong City Mandal...,121.056831,14.581404,Seafood Asian Filipino Indian,...,Botswana PulaP,Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,ooma,162,mandaluyongcity,Third Floor Mega Fashion Hall SM Megamall Orti...,SM Megamall Ortigas Mandaluyong City,SM Megamall Ortigas Mandaluyong City Mandaluyo...,121.056475,14.585318,Japanese Sushi,...,Botswana PulaP,No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,sambokojin,162,mandaluyongcity,Third Floor Mega Atrium SM Megamall Ortigas Ma...,SM Megamall Ortigas Mandaluyong City,SM Megamall Ortigas Mandaluyong City Mandaluyo...,121.057508,14.584450,Japanese Korean,...,Botswana PulaP,Yes,No,No,No,4,4.8,Dark Green,Excellent,229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9542,5926979,lemank_lt_r,208,stanbul,Caferaa Mahallesi Neet _mer Sokak No 9A Kad۱k_...,Kad۱k_y Merkez,Kad۱k_y Merkez stanbul,29.022805,40.989705,Restaurant Cafe,...,Turkish LiraTL,No,No,No,No,3,3.7,Yellow,Good,506
9544,5915547,karak_yg_ll_olu,208,stanbul,Kemanke Karamustafa Paa Mahallesi R۱ht۱m Cadde...,Karak_y,Karak_y stanbul,28.977636,41.022904,Desserts B_rek,...,Turkish LiraTL,No,No,No,No,2,4.7,Dark Green,Excellent,1305
9545,5915054,baltazar,208,stanbul,Kemanke Karamustafa Paa Mahallesi K۱l۱_ Ali Pa...,Karak_y,Karak_y stanbul,28.981103,41.025785,Burger Izgara,...,Turkish LiraTL,No,No,No,No,3,4.3,Green,Very Good,870
9547,5908749,cevizaac۱,208,stanbul,Kouyolu Mahallesi Muhittin st_nda Caddesi No 8...,Kouyolu,Kouyolu stanbul,29.041297,41.009847,World Cuisine Patisserie Cafe,...,Turkish LiraTL,No,No,No,No,3,4.2,Green,Very Good,1034


In [21]:
df.columns

Index(['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address',
       'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines',
       'Average Cost for two', 'Currency', 'Has Table booking',
       'Has Online delivery', 'Is delivering now', 'Switch to order menu',
       'Price range', 'Aggregate rating', 'Rating color', 'Rating text',
       'Votes'],
      dtype='object')

In [22]:
df_new = df.drop(columns= ["Restaurant ID", "Country Code", "Address", "Locality", "Locality Verbose", 'Longitude', 'Latitude','Average Cost for two', 'Currency', 'Has Table booking',
       'Has Online delivery', 'Is delivering now', 'Switch to order menu', 'Rating color', 'Rating text',
       'Votes',"Aggregate rating"],axis=1)

In [23]:
df_new

,Restaurant Name,City,Cuisines,Price range
0,lepetitsouffle,makaticity,French Japanese Desserts,3
1,izakayakikufuji,makaticity,Japanese,3
2,heatedsashangrila,mandaluyongcity,Seafood Asian Filipino Indian,4
3,ooma,mandaluyongcity,Japanese Sushi,4
4,sambokojin,mandaluyongcity,Japanese Korean,4
...,...,...,...,...
9542,lemank_lt_r,stanbul,Restaurant Cafe,3
9544,karak_yg_ll_olu,stanbul,Desserts B_rek,2
9545,baltazar,stanbul,Burger Izgara,3
9547,cevizaac۱,stanbul,World Cuisine Patisserie Cafe,3


In [24]:
df_new.columns

Index(['Restaurant Name', 'City', 'Cuisines', 'Price range'], dtype='object')

In [25]:
df_new['data'] = df_new[df_new.columns[1:]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1
)

In [26]:
df_new["data"]

0                 makaticity French Japanese Desserts 3
1                                 makaticity Japanese 3
2       mandaluyongcity Seafood Asian Filipino Indian 4
3                      mandaluyongcity Japanese Sushi 4
4                     mandaluyongcity Japanese Korean 4
                             ...                       
9542                          stanbul Restaurant Cafe 3
9544                           stanbul Desserts B_rek 2
9545                            stanbul Burger Izgara 3
9547            stanbul World Cuisine Patisserie Cafe 3
9548                    stanbul Italian World Cuisine 4
Name: data, Length: 1826, dtype: object

In [27]:
vector = CountVectorizer()

vectorized_data = vector.fit_transform(df_new["data"])

In [28]:
vectorized_data

<1826x271 sparse matrix of type '<class 'numpy.int64'>'
	with 9002 stored elements in Compressed Sparse Row format>

In [29]:
similiar = cosine_similarity(vectorized_data)

In [30]:
print(similiar)

[[1.         0.70710678 0.         ... 0.         0.         0.        ]
 [0.70710678 1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.25819889 0.28867513]
 [0.         0.         0.         ... 0.25819889 1.         0.67082039]
 [0.         0.         0.         ... 0.28867513 0.67082039 1.        ]]


In [31]:
df_vector = pd.DataFrame(similiar,columns=df['Cuisines'], index=df['Cuisines']).reset_index()

df_vector

Cuisines,Cuisines,French Japanese Desserts,Japanese,Seafood Asian Filipino Indian,Japanese Sushi,Japanese Korean,Chinese,Asian European,Seafood Filipino Asian European,European Asian Indian,...,Kebab Turkish Pizza D_ner,Turkish Pizza,Kebab Desserts Turkish Pizza,Restaurant Cafe Desserts,Restaurant Cafe Turkish Desserts,Restaurant Cafe,Desserts B_rek,Burger Izgara,World Cuisine Patisserie Cafe,Italian World Cuisine
0,French Japanese Desserts,1.000000,0.707107,0.000000,0.288675,0.288675,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.223607,0.250000,0.223607,0.000000,0.288675,0.000000,0.000000,0.000000
1,Japanese,0.707107,1.000000,0.000000,0.408248,0.408248,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Seafood Asian Filipino Indian,0.000000,0.000000,1.000000,0.258199,0.258199,0.316228,0.258199,0.6,0.447214,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Japanese Sushi,0.288675,0.408248,0.258199,1.000000,0.666667,0.408248,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Japanese Korean,0.288675,0.408248,0.258199,0.666667,1.000000,0.408248,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,Restaurant Cafe,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.866025,0.774597,1.000000,0.333333,0.333333,0.516398,0.288675
1822,Desserts B_rek,0.288675,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.258199,0.577350,0.516398,0.333333,1.000000,0.333333,0.258199,0.288675
1823,Burger Izgara,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.288675,0.258199,0.333333,0.333333,1.000000,0.258199,0.288675
1824,World Cuisine Patisserie Cafe,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.447214,0.400000,0.516398,0.258199,0.258199,1.000000,0.670820


In [34]:
input_cuisines = 'Japanese'
recommendations = pd.DataFrame(df_vector.nlargest(11,input_cuisines)['Cuisines'])
recommendations = recommendations[recommendations['Cuisines']!=input_cuisines]
indexes = recommendations.index
indexes = indexes.tolist()
df_rec = pd.DataFrame({
    "Restaurant Name" : [],
    "Cuisines" : [],
    "Price range" : []
})
i = 0
for index in indexes:
    new_row = pd.DataFrame({
        "Restaurant Name" : [df_temp.loc[int(index), "Restaurant Name"]],
        "Cuisines" : [recommendations.values[i]],
        "Price range" : [df_temp.loc[int(index), "Price range"]]
    })
    df_rec = pd.concat([df_rec, new_row], ignore_index= True)
    i = i+1

recommendations

,Cuisines
0,French Japanese Desserts
3,Japanese Sushi
4,Japanese Korean
50,Sushi Japanese
125,Asian Japanese
129,Japanese Steak
959,Japanese Asian
1057,Chinese Japanese
1289,Japanese French
1318,Thai Japanese


In [35]:
df_rec

,Restaurant Name,Cuisines,Price range
0,Le Petit Souffle,[French Japanese Desserts],3.0
1,Ooma,[Japanese Sushi],4.0
2,Sambo Kojin,[Japanese Korean],4.0
3,Garota de Ipanema,[Sushi Japanese],4.0
4,The Bee's Knees,[Asian Japanese],2.0
5,Sconyers Bar B Que,[Japanese Steak],2.0
6,Subway,[Japanese Asian],2.0
7,Aravalli Owls,[Chinese Japanese],1.0
8,Sibang Bakery,[Japanese French],1.0
9,Rawleaf,[Thai Japanese],2.0
